<a href="https://colab.research.google.com/github/rtajeong/M3_new_2025/blob/main/M3_lab53_naver_movie_rev2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

네이버영화평점
==
- 감성분석
- 네이버 영화평점 (Naver sentiment movie corpus v.1.0) 데이터(https://github.com/e9t/nsmc)
- 영화 리뷰 20만건이 저장됨. 각 평가 데이터는 0(부정), 1(긍정)으로 label 됨.

### 한글 자연어 처리
- KoNLPy(“코엔엘파이”라고 읽습니다)는 한국어 정보처리를 위한 파이썬 패키지입니다.
- konlpy 패키지에서 제공하는 Twitter라는 문서 분석 라이브러리 사용 (트위터 분석 뿐 아니라 한글 텍스트
  처리도 가능)
- colab 사용 권장

# 로지스틱회귀를 이용한 감성분석

In [1]:
!pip install konlpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 19.4/19.4 MB 72.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 495.9/495.9 kB 35.7 MB/s eta 0:00:00


In [2]:
# Curl:
# curl is a tool to transfer data from or to a server, using one of the supported protocols (HTTP, HTTPS, FTP,
# FTPS, SCP, SFTP, TFTP, DICT, TELNET, LDAP or FILE). The command is designed to work without user interaction.
#
# curl -L : (HTTP/HTTPS) If the server reports that the requested page has moved to a different location
# (indicated with a Location: header and a 3XX response code), this option will make curl redo the request
# on the new place.

In [3]:
# 네이버 영화 평점 데이터 다운로드
!curl -L https://bit.ly/2X9Owwr -o ratings_train.txt
!curl -L https://bit.ly/2WuLd5I -o ratings_test.txt

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   118  100   118    0     0    540      0 --:--:-- --:--:-- --:--:--   541
100   131  100   131    0     0    282      0 --:--:-- --:--:-- --:--:--   282
100    17  100    17    0     0     19      0 --:--:-- --:--:-- --:--:--     0
100 14.0M  100 14.0M    0     0  4262k      0  0:00:03  0:00:03 --:--:-- 9227k
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   117  100   117    0     0    403      0 --:--:-- --:--:-- --:--:--   403
100   130  100   130    0     0    195      0 --:--:-- --:--:-- --:--:--   195
100    17  100    17    0     0     16      0  0:00:01  0:00:01 --:--:--    16
100 4827k  100 4827k    0     0  1407k      0  0:00:03  0:00:03 --:--:-- 2183k


In [4]:
import konlpy
import pandas as pd
from konlpy.tag import Twitter, Okt
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
# from sklearn.pipeline import make_pipeline
# import pickle
# import os.path

# 데이터 로드
# keep_default_na = False: use an empty string instead of a NaN when parsing missing values

df_train = pd.read_csv('ratings_train.txt', delimiter='\t', keep_default_na=False)
df_test = pd.read_csv('ratings_test.txt', delimiter='\t', keep_default_na=False)
pd.concat([df_train.head(), df_train.tail()])

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1
149995,6222902,인간이 문제지.. 소는 뭔죄인가..,0
149996,8549745,평점이 너무 낮아서...,1
149997,9311800,이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?,0
149998,2376369,청춘 영화의 최고봉.방황과 우울했던 날들의 자화상,1
149999,9619869,한국 영화 최초로 수간하는 내용이 담긴 영화,0


In [5]:
df_test.head()

,id,document,label
0,6270596,굳 ㅋ,1
1,9274899,GDNTOPCLASSINTHECLUB,0
2,8544678,뭐야 이 평점들은.... 나쁘진 않지만 10점 짜리는 더더욱 아니잖아,0
3,6825595,지루하지는 않은데 완전 막장임... 돈주고 보기에는....,0
4,6723715,3D만 아니었어도 별 다섯 개 줬을텐데.. 왜 3D로 나와서 제 심기를 불편하게 하죠??,0


In [6]:
df_train.dtypes, df_test.dtypes

(id           int64
 document    object
 label        int64
 dtype: object,
 id           int64
 document    object
 label        int64
 dtype: object)

In [7]:
df_train.shape, df_test.shape

((150000, 3), (50000, 3))

In [8]:
text_train, y_train = df_train['document'].values, df_train['label'].values
text_test, y_test = df_test['document'].values, df_test['label'].values

In [9]:
text_train.shape, text_test.shape   # too big

((150000,), (50000,))

In [10]:
text_train, y_train = text_train[:2000], y_train[:2000]
text_test, y_test = text_test[:1000], y_test[:1000]

In [11]:
text_train.shape, text_test.shape

((2000,), (1000,))

In [12]:
text_train[0]

'아 더빙.. 진짜 짜증나네요 목소리'

In [13]:
from konlpy.tag import Twitter, Okt

def okt_tokenizer(text):
    return Okt().morphs(text)


In [14]:
cv = CountVectorizer(tokenizer=okt_tokenizer, max_features = 500, min_df=10)
x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)

lr = LogisticRegression()
lr.fit(x_train,y_train)

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


LogisticRegression()

In [15]:
len(cv.vocabulary_), x_train.shape, x_test.shape

(388, (2000, 388), (1000, 388))

In [16]:
y_train[:5], y_test[:5]

(array([0, 1, 0, 0, 1]), array([1, 0, 0, 0, 0]))

In [17]:
print("훈련 데이터 점수 : ", lr.score(x_train, y_train))
print("테스트 데이터 점수 : ", lr.score(x_test, y_test))

훈련 데이터 점수 :  0.811
테스트 데이터 점수 :  0.712


In [18]:
feature_names = cv.get_feature_names_out()
print(feature_names[:100])

['!' '!!' '!!!' "'" '(' ')' ',' ',,' '-' '.' '..' '...' '....' '.....' '0'
 '1' '10' '2' '3' '4' '5' ';' ';;' '?' '??' '^^' 'oo' 'tv' '~' '~~' 'ㅋ'
 'ㅋㅋ' 'ㅋㅋㅋ' 'ㅎ' 'ㅎㅎ' 'ㅜㅜ' 'ㅠ' 'ㅠㅠ' 'ㅡㅡ' '가' '가슴' '가장' '가족' '감' '감독' '감동'
 '감정' '같다' '같은' '같은데' '개' '개봉' '거' '건' '건가' '건지' '걸' '것' '게' '결국' '결말'
 '계속' '고' '공감' '과' '굿' '그' '그냥' '그래도' '그런' '그리고' '그만' '극' '극장' '근데' '급'
 '기' '기대' '기분' '기억' '긴장감' '까지' '꼭' '꽤' '끝' '나' '나름' '난' '남' '남자' '내' '내내'
 '내용' '냐' '너' '너무' '넘' '네' '눈' '눈물']


# 불용어 처리
- 한국어  불용어 확인은 형태소 분석 라이브러리인 KoLPy 를 이용하면 됨.
- (예) 한국어 품사 중 조사를 추출하는 예
- pos (part-of-speech): 품사 (명사, 동사, ...)

In [19]:
Okt().morphs("텍스트 데이터를 이용한 불용어 사전을 구축")

['텍스트', '데이터', '를', '이용', '한', '불', '용어', '사전', '을', '구축']

In [20]:
Okt().pos("텍스트 데이터를 이용한 불용어 사전을 구축")

[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('한', 'Josa'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사전', 'Noun'),
 ('을', 'Josa'),
 ('구축', 'Noun')]

In [21]:
Okt().pos("텍스트 데이터를 이용한 불용어 사전을 구촉", norm=True)

[('텍스트', 'Noun'),
 ('데이터', 'Noun'),
 ('를', 'Josa'),
 ('이용', 'Noun'),
 ('한', 'Josa'),
 ('불', 'Noun'),
 ('용어', 'Noun'),
 ('사전', 'Noun'),
 ('을', 'Josa'),
 ('구', 'Modifier'),
 ('촉', 'Noun')]

In [22]:
Okt().nouns("텍스트 데이터를 이용한 불용어 사전을 구축")

['텍스트', '데이터', '이용', '불', '용어', '사전', '구축']

- norm: 오타수정, stem: 어근 찾기

In [23]:
from konlpy.tag import Okt
okt = Okt()
word_tags = okt.pos("텍스트 데이터를 이용한 불용어 사전을 구축, ~ ! == ", norm=True, stem=True)
print(word_tags)
words = [word[0] for word in word_tags if word[1]=="Noun"]
print (words)

[('텍스트', 'Noun'), ('데이터', 'Noun'), ('를', 'Josa'), ('이용', 'Noun'), ('한', 'Josa'), ('불', 'Noun'), ('용어', 'Noun'), ('사전', 'Noun'), ('을', 'Josa'), ('구축', 'Noun'), (',', 'Punctuation'), ('~', 'Punctuation'), ('!', 'Punctuation'), ('==', 'Punctuation')]
['텍스트', '데이터', '이용', '불', '용어', '사전', '구축']


In [24]:
def okt_tokenizer2(text):
    word_tags = okt.pos(text, norm=True, stem=True)
    words = [word[0] for word in word_tags if word[1]!="Josa" and word[1]!="Punctuation"]
    # words = [word[0] for word in word_tags if word[1]=="Noun"]
    return words

In [25]:
cv = CountVectorizer(tokenizer=okt_tokenizer2, max_features = 1000, min_df=10)
x_train = cv.fit_transform(text_train)
x_test = cv.transform(text_test)

lr = LogisticRegression()
lr.fit(x_train,y_train)
print("훈련 데이터 점수 : ", lr.score(x_train, y_train))
print("테스트 데이터 점수 : ", lr.score(x_test, y_test))

/usr/local/lib/python3.12/dist-packages/sklearn/feature_extraction/text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


훈련 데이터 점수 :  0.8195
테스트 데이터 점수 :  0.733


In [26]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(max_depth=30)
rf.fit(x_train,y_train)
print("훈련 데이터 점수 : ", rf.score(x_train, y_train))
print("테스트 데이터 점수 : ", rf.score(x_test, y_test))

훈련 데이터 점수 :  0.899
테스트 데이터 점수 :  0.722


In [27]:
from sklearn.svm import SVC
sv = SVC()
sv.fit(x_train,y_train)
print("훈련 데이터 점수 : ", sv.score(x_train, y_train))
print("테스트 데이터 점수 : ", sv.score(x_test, y_test))

훈련 데이터 점수 :  0.8905
테스트 데이터 점수 :  0.739


# Exercise